In [37]:
# Import libs
import geemap, ee
import geemap.colormaps as cm
# initialize our map
Map = geemap.Map()

In [38]:
# get our Ho Chi Minh, Vietnam
# I have taken level 1 data for country data from FAO datasets
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1") \
   .filter(ee.Filter.eq('ADM1_NAME',"Ho Chi Minh City")).geometry() 

In [39]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8').divide(10000),
            'RED': image.select('B4').divide(10000),
            'BLUE': image.select('B2').divide(10000)
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

In [40]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [41]:
# filter sentinel 2 images and apply the EVI formula, finally we obtain 
# single image with median operation

Sentinel_data = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate("2023-03-01","2023-03-31").filterBounds(aoi) \
    .map(getEVI).map(addDate).median()

In [42]:
# set some thresholds
#  A nice EVI palette
palett = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']

pall = {"min":0.1, "max":0.8, 'palette':palett}



Map.centerObject(aoi)
Map.addLayer(Sentinel_data.clip(aoi).select('EVI'), pall, "EVI")

Map.addLayerControl()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…